<a href="https://colab.research.google.com/github/alexisdr/uned-tfg/blob/main/UNED-TFG-2-datos-tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de datos de pruebas

Se toman los actos clínicos con mayor número de EAs y algunos sin EA para generar un subconjuto de pruebas.

# Parameters

*  GoldStandardTrainingEAs.txt: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
*  GoldStandardTestEAs.txt: equivalente al anterior pero con los datos del conjunto de pruebas.
*  Training: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
*  Training: equivalente a training pero con el conjunto de pruebas.



In [ ]:
base_path = '/drive/My Drive/CorpusPFG/'

#Archivos y rutas de los datos del CORPUS
GoldStandardTrainingEA_data_location = base_path + 'GoldStandardTrainingEAs.txt'
GoldStandardTestEA_data_location = base_path + 'GoldStandardTestEAs.txt'
training_files_location = base_path + 'Training/'
test_files_location = base_path + 'Test/'

#Archivos y rutas del subconjunto de pruebas
tiny_training_files_location = base_path + 'Tiny-Training/'
tiny_test_files_location = base_path + 'Tiny-Test/'
tiny_GoldStandardTrainingEA_data_location = base_path + 'Tiny-GoldStandardTrainingEAs.txt'

Se realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


# EAs disponibles en los casos clínicos

Carga del archivo de texto GoldStandardTrainingEAs.txt y GoldStandardTestEAs.txt en un data set

In [ ]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(GoldStandardTrainingEA_data_location, sep=' ', header=None, names=column_names)
df_goldStandardTestEAs = pd.read_csv(GoldStandardTestEA_data_location, sep=' ', header=None, names=column_names)

Se unen todos los códigos EAs de todas las columnas para conocer cuántos códigos EAs hay representados en los casos clínicos.

In [ ]:
ds_cantidad_TieneEA_EA1 = df_goldStandardTrainingEAs.TieneEA_EA1.value_counts(dropna=True, sort=True)
ds_cantidad_EA2 = df_goldStandardTrainingEAs.EA2.value_counts(dropna=True, sort=True)
ds_all = ds_cantidad_TieneEA_EA1.append(ds_cantidad_EA2)
ds_cantidad_EA3 = df_goldStandardTrainingEAs.EA3.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA3)
ds_cantidad_EA4 = df_goldStandardTrainingEAs.EA4.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA4)
ds_cantidad_EA5 = df_goldStandardTrainingEAs.EA5.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA5)
ds_cantidad_EA6 = df_goldStandardTrainingEAs.EA6.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA6)
ds_cantidad_EA7 = df_goldStandardTrainingEAs.EA7.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA7)
ds_cantidad_EA8 = df_goldStandardTrainingEAs.EA8.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA8)
ds_cantidad_EA9 = df_goldStandardTrainingEAs.EA9.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA9)

df_cantidad = pd.DataFrame(ds_all)
df_cantidad = df_cantidad.reset_index()
df_cantidad.columns = ['eas', 'counts'] 
df_cantidad.count()

<ipython-input-4-ffaff8709537>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds_all = ds_cantidad_TieneEA_EA1.append(ds_cantidad_EA2)
<ipython-input-4-ffaff8709537>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds_all = ds_all.append(ds_cantidad_EA3)
<ipython-input-4-ffaff8709537>:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds_all = ds_all.append(ds_cantidad_EA4)
<ipython-input-4-ffaff8709537>:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds_all = ds_all.append(ds_cantidad_EA5)
<ipython-input-4-ffaff8709537>:11: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. U

eas       789
counts    789
dtype: int64

# Generación de ficheros Tiny
Obtener las casos clínicos con EAs más repetidos para crear un dataset 
balanceado y hacer pruebas.

In [ ]:
df_cantidad.query("eas != 'NONE'").head(8)["eas"]

1    T38.0X5A
2    T45.515A
3    T50.2X5A
4         Y95
5       P01.1
6    T81.4XXA
7    T45.1X5A
8       Y83.1
Name: eas, dtype: object

Vamos a tomar los casos clínicos en los que aparezcan los códigos EAs de los 8 primeros del listado anterior.

In [ ]:
ea_count = 0
for count in df_cantidad.query("eas != 'NONE'").head(8)["counts"]:
  ea_count = ea_count + count
print("Cantidad de EAs de los 8 primeros: %s" % ea_count)

Cantidad de EAs de los 8 primeros: 711


Toma la lista de ficheros de origen

In [ ]:
import shutil
import os

def get_actos_list(rows):
  actos = df_goldStandardTrainingEAs.query("Acto == 0")
  for ea in rows["eas"]:
    actos = actos.append(get_actos(ea), ignore_index=True)
  return actos

def get_actos(EA_code):
  query = ("TieneEA_EA1 == '{EA}'")
  query = query.replace ('{EA}', EA_code)
  return df_goldStandardTrainingEAs.query(query)

def get_actos_file_list(actos):
  result = list()
  train_file_list = [f for f in os.listdir(training_files_location) 
                    if os.path.isfile(os.path.join(training_files_location, f))]
  for index, row in actos.iterrows():
    texto_filtro = str(row['Acto']) + '-'
    result.append(list(filter(lambda x: texto_filtro in x, train_file_list)))
  return result

def copy_files(actos_file_list):
  for file_list in actos_file_list:
    for file in file_list:
      shutil.copyfile (training_files_location + file, 
                       tiny_training_files_location + file)

Prepara las carpetas

In [ ]:
#shutil.rmtree(tiny_training_files_location)
#os.mkdir(tiny_training_files_location)

Copia los ficheros con código EA más relevantes

In [ ]:
#Obtiene la lista de actos con código EA
actos_list_eas = get_actos_list(df_cantidad.query("eas != 'NONE'").head(8))
#Obtiene la lista de ficheros de los actos con código EA
#actos_file_list = get_actos_file_list(actos_list_eas)
#Copia los ficheros a las rutas tiny
#copy_files(actos_file_list)

<ipython-input-7-05c0550e8e17>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actos = actos.append(get_actos(ea), ignore_index=True)


Copia los ficheros sin código EA


In [ ]:
#Obtiene la lista de actos sin código EA
actos_list_none = df_goldStandardTrainingEAs.query("TieneEA_EA1 == 'NONE'").head(100)
#Obtiene la lista de ficheros de los actos sin código EA
#actos_file_list = get_actos_file_list(actos_list_none)
#Copia los ficheros a las rutas tiny
#copy_files(actos_file_list)

In [ ]:
import os
from os import path

if (path.exists(tiny_GoldStandardTrainingEA_data_location)):
  os.remove(tiny_GoldStandardTrainingEA_data_location)

actos_list = actos_list_none.append(actos_list_eas, ignore_index=True)
actos_list.to_csv(tiny_GoldStandardTrainingEA_data_location, header=None, index=None, sep=' ', mode='a')

actos_list.TieneEA_EA1.unique()

<ipython-input-13-7f5ddbb3b8fe>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actos_list = actos_list_none.append(actos_list_eas, ignore_index=True)


array(['NONE', 'T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1',
       'T81.4XXA', 'T45.1X5A', 'Y83.1'], dtype=object)

In [ ]:
tiny_GoldStandardTrainingEA_data_location

'/drive/My Drive/CorpusPFG/Tiny-GoldStandardTrainingEAs.txt'